In [ ]:
import pandas as pd
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/train.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
train.head()

In [ ]:
train["Target"].value_counts()/(train["Target"].value_counts().sum())

In [ ]:
train.shape

In [ ]:
train.isnull().sum()[train.isnull().sum()>0]

In [ ]:
train.isnull().sum()[train.isnull().sum()>0]/(train["Target"].value_counts().sum())

The "v2a1", "v18q1" and "rez_esc" columns have more than 70% missing values. Because of that they will be dropped from the table.

In [ ]:
train2 = train.drop(["v2a1", "v18q1", "rez_esc"], axis=1)
train2.isnull().sum()[train.isnull().sum()>0]

The remaining lines with missing data will be removed

In [ ]:
ntrain = train2.dropna();
ntrain

Now we need to assign numerical values to the categorical variables

In [ ]:
numTrain = ntrain.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
Xtrain = numTrain.iloc[:,1:139]
Ytrain = numTrain.Target

Now it's time to select the value of k. This will be done using for loops. 

In [ ]:
mscores = []
mscores2 = []
mscores3 = []
for x in range (1, 11):
    knn = KNeighborsClassifier(n_neighbors=(x*100))
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)
    mscores.append(scores.mean())
for x in range (1, 11):
    knn = KNeighborsClassifier(n_neighbors=(x*10 + mscores.index(max(mscores))*100))
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)
    mscores2.append(scores.mean())
for x in range (1, 11):
    knn = KNeighborsClassifier(n_neighbors=(x + mscores2.index(max(mscores2))*10))
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)
    mscores3.append(scores.mean())
max(mscores3)

In [ ]:
mscores.index(max(mscores))*100 + mscores2.index(max(mscores2))*10 + mscores3.index(max(mscores3)) + 1

And now we will train our data

In [ ]:
test = pd.read_csv("../input/test.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
test2 = test.drop(["v2a1", "v18q1", "rez_esc"], axis=1)
ntest = test2.dropna()
numTest = ntest.apply(preprocessing.LabelEncoder().fit_transform)
Xtest= numTest.iloc[:,1:139]
knn.fit(Xtrain,Ytrain)
Ytest = knn.predict(Xtest)
Ytest

In [ ]:
Ytabela = pd.DataFrame(index=ntest.Id,columns=['income'])
Ytabela['income'] = Ytest
Ytabela

In [ ]:
Ytabela.to_csv('prediction.csv')